## 📦 Step 1: Install Required Packages

In [ ]:
# Install packages (run once)
!pip install --upgrade pandas yfinance tensorflow tensorflow-probability tqdm numpy xlsxwriter -q

## 💡 Alternative: If yfinance fails completely

If you're experiencing persistent yfinance errors, you can:
1. Use Google Sheets data (like your original code)
2. Download from SET website and upload CSV
3. Use alternative APIs (Alpha Vantage, Tiingo, etc.)

Uncomment the cell below to use Google Sheets data instead:

## 📚 Step 2: Import Libraries and Configure GPU

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import itertools
import time
import yfinance as yf
from typing import List, Tuple, Dict, Optional
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Configure GPU (Enable memory growth to avoid OOM errors)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU Available: {len(gpus)} device(s)")
        for i, gpu in enumerate(gpus):
            print(f"   GPU {i}: {gpu.name}")
    except RuntimeError as e:
        print(f"⚠️ GPU Configuration Error: {e}")
else:
    print("ℹ️ No GPU found. Using CPU (will be slower)")

print(f"\n🔧 TensorFlow Version: {tf.__version__}")
print(f"🔧 Numpy Version: {np.__version__}")
print(f"🔧 Pandas Version: {pd.__version__}")

## 🎯 Step 3: Ultra-Fast Vectorized Portfolio Calculations

In [ ]:
@tf.function(reduce_retracing=True)
def calculate_portfolio_metrics_vectorized(P0: tf.Tensor, Wi: tf.Tensor, fund: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
    """
    Ultra-optimized vectorized calculation using TensorFlow.
    10-50x faster than traditional numpy/pandas loops.
    
    Args:
        P0: Stock prices over time [time_steps, n_stocks]
        Wi: Portfolio weights [n_samples, n_stocks]
        fund: Investment fund amount (scalar)
    
    Returns:
        avg_return: Average portfolio return [n_samples]
        volatility: Portfolio volatility using log returns [n_samples]
        variance: Portfolio variance [n_samples]
    """
    # Calculate number of shares for each portfolio
    # X0 = Wi * fund / P0[0]  -> [n_samples, n_stocks]
    X0 = Wi * fund / P0[0]
    
    # Calculate portfolio value over time
    # Port_Value = X0 @ P0.T  -> [n_samples, time_steps]
    Port_Value = tf.matmul(tf.cast(X0, dtype=tf.float64), tf.transpose(P0))
    
    # Portfolio return (percentage)
    Port_return = (Port_Value - fund) / fund * 100.0
    
    # Average return across time periods
    avg_return = tf.reduce_mean(Port_return, axis=1)
    
    # Variance calculation
    variance = tf.math.reduce_variance(Port_return, axis=1)
    
    # Log returns calculation (vectorized, no pandas shift needed)
    epsilon = 1e-10  # Avoid log(0)
    log_Port_Value = tf.math.log(Port_Value + epsilon)
    
    # Calculate log return differences (daily log returns)
    log_return = log_Port_Value - tf.roll(log_Port_Value, shift=1, axis=1)
    
    # Calculate difference of log returns (second-order difference)
    df_log = log_return - tf.roll(log_return, shift=2, axis=1)
    
    # Skip first 2 time steps (affected by roll operation)
    df_log_valid = df_log[:, 2:]
    
    # Sum of squared log differences
    sum_log = tf.reduce_sum(tf.square(df_log_valid), axis=1)
    
    # Volatility (realized volatility)
    volatility = tf.sqrt(sum_log + epsilon)
    
    return avg_return, volatility, variance

## 🏗️ Step 4: Portfolio Optimizer Class (Supports N=2, N=3, N=5)

In [ ]:
class UltraFastPortfolioOptimizer:
    """
    Ultra-fast portfolio optimizer with batch processing and GPU acceleration.
    Supports multiple portfolio sizes (N=2, N=3, N=5) and multiple return targets.
    """
    
    def __init__(self, 
                 df: pd.DataFrame, 
                 companies: np.ndarray, 
                 Wi: tf.Tensor, 
                 fund: float, 
                 n_stocks: int,
                 batch_size: int = 100):
        """
        Initialize optimizer.
        
        Args:
            df: DataFrame with stock prices (from yfinance)
            companies: Array of stock combinations
            Wi: Weight samples tensor [n_samples, n_stocks]
            fund: Investment fund amount
            n_stocks: Number of stocks per portfolio (2, 3, or 5)
            batch_size: Batch size for processing (adjust based on memory)
        """
        self.df = df
        self.companies = companies
        self.Wi_tensor = tf.constant(Wi.numpy() if hasattr(Wi, 'numpy') else Wi, dtype=tf.float64)
        self.fund_tensor = tf.constant(fund, dtype=tf.float64)
        self.fund = float(fund)
        self.n_stocks = n_stocks
        self.batch_size = batch_size
        
        print(f"\n{'='*80}")
        print(f"🚀 Ultra-Fast Portfolio Optimizer Initialized")
        print(f"{'='*80}")
        print(f"📊 Portfolio Size (N): {n_stocks}")
        print(f"🎲 Weight Samples: {len(self.Wi_tensor):,}")
        print(f"🏢 Stock Combinations: {len(companies):,}")
        print(f"💰 Investment Fund: {fund:,.0f} THB")
        print(f"📦 Batch Size: {batch_size}")
        print(f"💻 Device: {'GPU' if gpus else 'CPU'}")
        print(f"{'='*80}\n")
    
    def process_batch(self, company_batch_indices: List[int]) -> List[Dict]:
        """Process a batch of company combinations."""
        batch_results = []
        
        for idx in company_batch_indices:
            company_tuple = self.companies[idx]
            
            try:
                # Get price data for this combination
                if 'Close' in self.df.columns.names:
                    A = self.df['Close'][list(company_tuple)].values
                else:
                    A = self.df[list(company_tuple)].values
                
                # Check for invalid data
                if np.any(np.isnan(A)) or A.shape[0] == 0:
                    continue
                
                # Convert to tensor
                P0 = tf.constant(A, dtype=tf.float64)
                P0 = tf.reshape(P0, [-1, self.n_stocks])
                
                # Calculate metrics using vectorized function
                avg_return, volatility, variance = calculate_portfolio_metrics_vectorized(
                    P0, self.Wi_tensor, self.fund_tensor
                )
                
                batch_results.append({
                    'idx': idx,
                    'companies': company_tuple,
                    'avg_return': avg_return.numpy(),
                    'volatility': volatility.numpy(),
                    'variance': variance.numpy()
                })
                
            except Exception as e:
                # Skip invalid combinations
                continue
        
        return batch_results
    
    def optimize_multiple_returns(self, 
                                   return_targets: List[Tuple[float, float]], 
                                   max_combinations: Optional[int] = None) -> Tuple[Dict, Dict]:
        """
        Optimize for multiple return targets simultaneously.
        
        Args:
            return_targets: List of (min, max) return target tuples
                           e.g., [(0.9999, 1.0001), (1.9999, 2.0001), ...]
            max_combinations: Limit processing (for testing)
        
        Returns:
            best_portfolios: Dictionary with best portfolio for each target
            all_results: All valid results for each target
        """
        n_combinations = len(self.companies) if max_combinations is None else min(max_combinations, len(self.companies))
        
        # Initialize storage
        best_portfolios = {
            i: {
                'volatility': None,
                'variance': None,
                'weights': None,
                'companies': None,
                'avg_return': None,
                'target_return': (return_targets[i][0] + return_targets[i][1]) / 2
            }
            for i in range(len(return_targets))
        }
        
        all_results = {i: [] for i in range(len(return_targets))}
        
        # Calculate batch parameters
        n_batches = (n_combinations + self.batch_size - 1) // self.batch_size
        
        print(f"\n🔄 Starting Optimization...")
        print(f"📊 Total Combinations: {n_combinations:,}")
        print(f"📦 Number of Batches: {n_batches:,}")
        print(f"🎯 Return Targets: {len(return_targets)}")
        print(f"🔢 Total Calculations: {n_combinations * len(self.Wi_tensor):,}")
        print(f"\n{'='*80}\n")
        
        start_time = time.time()
        
        # Process batches with progress bar
        with tqdm(total=n_batches, desc="Processing", ncols=100) as pbar:
            for batch_idx in range(n_batches):
                batch_start = batch_idx * self.batch_size
                batch_end = min(batch_start + self.batch_size, n_combinations)
                
                # Process batch
                batch_indices = list(range(batch_start, batch_end))
                batch_results = self.process_batch(batch_indices)
                
                # Update best portfolios for each return target
                for result in batch_results:
                    avg_returns = result['avg_return']
                    volatilities = result['volatility']
                    variances = result['variance']
                    companies = result['companies']
                    
                    # Check each return target
                    for target_idx, (min_ret, max_ret) in enumerate(return_targets):
                        # Find portfolios meeting return constraint
                        mask = (avg_returns >= min_ret) & (avg_returns <= max_ret)
                        
                        if np.any(mask):
                            # Get best (minimum volatility)
                            masked_vol = volatilities[mask]
                            min_vol_idx = np.argmin(masked_vol)
                            min_vol = masked_vol[min_vol_idx]
                            
                            # Get corresponding data
                            masked_indices = np.where(mask)[0]
                            global_idx = masked_indices[min_vol_idx]
                            best_weights = self.Wi_tensor[global_idx].numpy()
                            best_return = avg_returns[global_idx]
                            best_variance = variances[global_idx]
                            
                            # Update if better than current best
                            current_best = best_portfolios[target_idx]
                            if current_best['volatility'] is None or min_vol < current_best['volatility']:
                                best_portfolios[target_idx] = {
                                    'volatility': float(min_vol),
                                    'variance': float(best_variance),
                                    'weights': best_weights,
                                    'companies': companies,
                                    'avg_return': float(best_return),
                                    'target_return': (min_ret + max_ret) / 2
                                }
                                all_results[target_idx].append(best_portfolios[target_idx].copy())
                
                # Update progress
                pbar.update(1)
                
                # Show intermediate results every 10% progress
                if (batch_idx + 1) % max(1, n_batches // 10) == 0:
                    elapsed = time.time() - start_time
                    progress = batch_end / n_combinations * 100
                    rate = batch_end / elapsed if elapsed > 0 else 0
                    eta = (n_combinations - batch_end) / rate if rate > 0 else 0
                    
                    pbar.set_postfix({
                        'Progress': f'{progress:.1f}%',
                        'Rate': f'{rate:.1f} comb/s',
                        'ETA': f'{eta/60:.1f}m'
                    })
        
        total_time = time.time() - start_time
        
        # Print final statistics
        print(f"\n{'='*80}")
        print(f"✅ Optimization Complete!")
        print(f"{'='*80}")
        print(f"⏱️  Total Time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
        print(f"⚡ Average Rate: {n_combinations/total_time:.1f} combinations/second")
        print(f"🎯 Portfolios Found:")
        for i, portfolio in best_portfolios.items():
            status = "✅ Found" if portfolio['volatility'] is not None else "❌ Not Found"
            print(f"   {i+1}% Return: {status}")
        print(f"{'='*80}\n")
        
        return best_portfolios, all_results

## 🎨 Step 5: Results Display and Allocation Functions

In [ ]:
def calculate_allocation(weights: np.ndarray, 
                        fund: float, 
                        prices: Dict[str, float]) -> Tuple[Dict, float, float]:
    """
    Calculate actual stock allocation and remaining cash.
    
    Args:
        weights: Portfolio weights
        fund: Investment fund
        prices: Dictionary of stock prices
    
    Returns:
        allocation: Dictionary with allocation details
        total_invested: Total amount invested
        remaining_cash: Remaining cash
    """
    allocation = {}
    total_invested = 0
    
    for stock, weight, price in zip(prices.keys(), weights, prices.values()):
        amount_to_invest = fund * weight
        shares = int(amount_to_invest / price)  # Buy whole shares only
        invested = shares * price
        
        allocation[stock] = {
            'weight': weight,
            'shares': shares,
            'price': price,
            'invested': invested
        }
        total_invested += invested
    
    remaining_cash = fund - total_invested
    
    return allocation, total_invested, remaining_cash


def display_results(best_portfolios: Dict, 
                   return_targets: List[Tuple[float, float]], 
                   df: pd.DataFrame, 
                   fund: float,
                   n_stocks: int):
    """
    Display optimization results in a beautiful format.
    """
    print("\n" + "="*90)
    print(f"{'🏆 PORTFOLIO OPTIMIZATION RESULTS':^90}")
    print(f"{'Portfolio Size (N): ' + str(n_stocks):^90}")
    print("="*90 + "\n")
    
    for target_idx, (min_ret, max_ret) in enumerate(return_targets):
        portfolio = best_portfolios[target_idx]
        target_return = (min_ret + max_ret) / 2
        
        print(f"\n{'─'*90}")
        print(f"🎯 TARGET RETURN: {target_return:.1f}%")
        print(f"{'─'*90}")
        
        if portfolio['volatility'] is None:
            print("\n❌ No portfolio found for this return target\n")
            continue
        
        companies = portfolio['companies']
        weights = portfolio['weights']
        
        print(f"\n📊 Portfolio Metrics:")
        print(f"   • Actual Return:    {portfolio['avg_return']:>10.4f}%")
        print(f"   • Volatility:       {portfolio['volatility']:>10.6f}")
        print(f"   • Variance:         {portfolio['variance']:>10.6f}")
        
        print(f"\n🏢 Selected Stocks & Allocation:")
        
        # Get latest prices
        latest_prices = {}
        for stock in companies:
            try:
                if 'Close' in df.columns.names:
                    latest_prices[stock] = df['Close'][stock].iloc[-1]
                else:
                    latest_prices[stock] = df[stock].iloc[-1]
            except:
                latest_prices[stock] = 0.0
        
        allocation, total_invested, remaining = calculate_allocation(
            weights, fund, latest_prices
        )
        
        # Display table header
        print(f"\n   {'Stock':<15} {'Weight':>10} {'Shares':>10} {'Price':>12} {'Invested':>14}")
        print(f"   {'-'*70}")
        
        # Display each stock
        for stock in companies:
            info = allocation[stock]
            print(f"   {stock:<15} {info['weight']:>9.4f} {info['shares']:>10} "
                  f"{info['price']:>12.2f} {info['invested']:>14,.2f}")
        
        print(f"\n💰 Investment Summary:")
        print(f"   • Total Fund:       {fund:>14,.2f} THB")
        print(f"   • Total Invested:   {total_invested:>14,.2f} THB ({total_invested/fund*100:>5.1f}%)")
        print(f"   • Remaining Cash:   {remaining:>14,.2f} THB ({remaining/fund*100:>5.1f}%)")
        print()


def save_results_to_file(best_portfolios: Dict, 
                         return_targets: List[Tuple[float, float]], 
                         n_stocks: int,
                         filename: str = "optimization_results.txt"):
    """
    Save optimization results to a text file.
    """
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("="*90 + "\n")
        f.write(f"PORTFOLIO OPTIMIZATION RESULTS (N={n_stocks})\n")
        f.write(f"Generated: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("="*90 + "\n\n")
        
        for target_idx, (min_ret, max_ret) in enumerate(return_targets):
            portfolio = best_portfolios[target_idx]
            target_return = (min_ret + max_ret) / 2
            
            f.write(f"\nTARGET RETURN: {target_return:.1f}%\n")
            f.write("-"*90 + "\n")
            
            if portfolio['volatility'] is not None:
                f.write(f"Companies: {', '.join(portfolio['companies'])}\n")
                f.write(f"Weights: {portfolio['weights']}\n")
                f.write(f"Volatility: {portfolio['volatility']:.6f}\n")
                f.write(f"Variance: {portfolio['variance']:.6f}\n")
                f.write(f"Actual Return: {portfolio['avg_return']:.4f}%\n")
            else:
                f.write("No portfolio found\n")
            f.write("\n")
    
    print(f"\n💾 Results saved to: {filename}")

## 🎬 Step 6: Main Execution - Configure Your Analysis

In [ ]:
# ============================================================================
# CONFIGURATION PARAMETERS - Adjust these for your analysis
# ============================================================================

# Portfolio configuration
N_STOCKS = 2  # Change to 2, 3, or 5
FUND = 500000  # Investment budget in THB
NUM_WEIGHT_SAMPLES = 200000  # Number of random weight combinations to test

# Data configuration
START_DATE = '2022-10-19'
END_DATE = '2022-11-30'

# Processing configuration
BATCH_SIZE = 100  # Adjust based on available memory (higher = faster but more RAM)
MAX_COMBINATIONS = None  # Set to a number (e.g., 1000) for testing, None for all

# Return targets (min, max) pairs
RETURN_TARGETS = [
    (0.9999, 1.0001),   # ~1% return
    (1.9999, 2.0001),   # ~2% return
    (2.9999, 3.0001),   # ~3% return
    (3.9999, 4.0001),   # ~4% return
    (4.9999, 5.0001),   # ~5% return
]

# Stock list (Thailand SET50)
STOCK_LIST = [
    "ADVANC.BK", "AOT.BK", "BBL.BK", "BDMS.BK", "BEM.BK", "BGRIM.BK",
    "BH.BK", "BJC.BK", "BTS.BK", "CBG.BK", "COM7.BK", "CPALL.BK",
    "CPF.BK", "CPN.BK", "CRC.BK", "DELTA.BK", "DTAC.BK", "EA.BK",
    "EGCO.BK", "GLOBAL.BK", "GPSC.BK", "GULF.BK", "HMPRO.BK", "INTUCH.BK",
    "IRPC.BK", "IVL.BK", "KBANK.BK", "KCE.BK", "KTB.BK", "KTC.BK",
    "LH.BK", "MINT.BK", "MTC.BK", "OR.BK", "OSP.BK", "PTT.BK",
    "PTTEP.BK", "PTTGC.BK", "RATCH.BK", "SAWAD.BK", "SCB.BK", "SCC.BK",
    "SCGP.BK", "STA.BK", "STGT.BK", "TISCO.BK", "TOP.BK", "TTB.BK",
    "TU.BK", "TRUE.BK"
]

print("\n" + "="*90)
print(f"{'⚙️  CONFIGURATION':^90}")
print("="*90)
print(f"\n📊 Portfolio Size (N): {N_STOCKS}")
print(f"💰 Investment Fund: {FUND:,.0f} THB")
print(f"🎲 Weight Samples: {NUM_WEIGHT_SAMPLES:,}")
print(f"📅 Date Range: {START_DATE} to {END_DATE}")
print(f"🏢 Number of Stocks: {len(STOCK_LIST)}")
print(f"📦 Batch Size: {BATCH_SIZE}")
print(f"🎯 Return Targets: {len(RETURN_TARGETS)} levels")
print("\n" + "="*90)

## 📥 Step 7: Download Stock Data

## 🎲 Step 8: Generate Weight Combinations

In [ ]:
print(f"\n🎲 Generating {NUM_WEIGHT_SAMPLES:,} random weight combinations...")
print(f"   (Weights sum to 1.0 using Dirichlet distribution)")

start_weights = time.time()
dist = tfp.distributions.Dirichlet(np.ones(N_STOCKS))
Wi = dist.sample(NUM_WEIGHT_SAMPLES)
weights_time = time.time() - start_weights

print(f"✅ Weight generation complete in {weights_time:.2f} seconds")
print(f"📊 Weight tensor shape: {Wi.shape}")
print(f"\n📋 Sample weights (first 5):")
print(Wi[:5].numpy())

## 🔄 Step 9: Generate Stock Combinations

In [ ]:
print(f"\n🔄 Generating all possible stock combinations (N={N_STOCKS})...")
print(f"   Using {len(STOCK_LIST)} available stocks")

start_combinations = time.time()
companies = np.array(list(itertools.combinations(STOCK_LIST, N_STOCKS)))
combinations_time = time.time() - start_combinations

print(f"\n✅ Combination generation complete in {combinations_time:.2f} seconds")
print(f"📊 Total combinations: {len(companies):,}")
print(f"💡 Formula: C({len(STOCK_LIST)},{N_STOCKS}) = {len(companies):,}")
print(f"\n📋 Sample combinations (first 5):")
for i, combo in enumerate(companies[:5]):
    print(f"   {i+1}. {', '.join(combo)}")

## 🚀 Step 10: Run Ultra-Fast Optimization

In [ ]:
print(f"\n📥 Downloading stock data for {len(STOCK_LIST)} stocks...")
print(f"📅 Period: {START_DATE} to {END_DATE}")
print(f"⚠️  Note: Some stocks may fail to download due to API limitations\n")

start_download = time.time()

# Download with error handling - try individual downloads if batch fails
try:
    # Try batch download first (faster)
    df_prices = yf.download(STOCK_LIST, start=START_DATE, end=END_DATE, progress=True, 
                           group_by='column', threads=True, ignore_tz=True)
    download_time = time.time() - start_download
    
    # Check if we got data
    if df_prices.empty:
        raise Exception("No data downloaded")
    
    print(f"\n✅ Batch download complete in {download_time:.2f} seconds")
    
except Exception as e:
    print(f"\n⚠️  Batch download failed: {e}")
    print(f"🔄 Trying individual stock downloads...\n")
    
    # Download stocks individually
    successful_stocks = []
    failed_stocks = []
    individual_data = {}
    
    for stock in tqdm(STOCK_LIST, desc="Downloading", ncols=80):
        try:
            stock_data = yf.download(stock, start=START_DATE, end=END_DATE, 
                                    progress=False, ignore_tz=True)
            if not stock_data.empty and 'Close' in stock_data.columns:
                individual_data[stock] = stock_data['Close']
                successful_stocks.append(stock)
            else:
                failed_stocks.append(stock)
        except Exception as ex:
            failed_stocks.append(stock)
    
    # Combine individual downloads
    if individual_data:
        df_prices = pd.DataFrame(individual_data)
        # Add MultiIndex for compatibility
        df_prices.columns = pd.MultiIndex.from_product([['Close'], df_prices.columns])
    else:
        raise Exception("Failed to download any stock data")
    
    download_time = time.time() - start_download
    
    print(f"\n✅ Individual download complete in {download_time:.2f} seconds")
    print(f"✅ Successfully downloaded: {len(successful_stocks)} stocks")
    if failed_stocks:
        print(f"⚠️  Failed to download: {len(failed_stocks)} stocks")
        print(f"   Failed stocks: {', '.join(failed_stocks[:10])}")
        if len(failed_stocks) > 10:
            print(f"   ... and {len(failed_stocks) - 10} more")
        
        # Update STOCK_LIST to only include successful stocks
        STOCK_LIST = successful_stocks
        print(f"\n📊 Continuing with {len(STOCK_LIST)} available stocks")

# Display summary
print(f"\n📊 Data shape: {df_prices.shape}")
print(f"📈 Trading days: {len(df_prices)}")

# Check for missing data
if 'Close' in df_prices.columns.names:
    close_prices = df_prices['Close']
    missing_pct = (close_prices.isnull().sum() / len(close_prices) * 100)
    stocks_with_missing = missing_pct[missing_pct > 0]
    if len(stocks_with_missing) > 0:
        print(f"\n⚠️  Stocks with missing data:")
        for stock, pct in stocks_with_missing.items():
            print(f"   {stock}: {pct:.1f}% missing")
        print(f"\n🔧 Will handle missing data during optimization")

# Display sample data
print(f"\n📋 Sample Data (first 3 rows, first 5 stocks):")
if 'Close' in df_prices.columns.names:
    print(df_prices['Close'].iloc[:3, :min(5, df_prices['Close'].shape[1])])
else:
    print(df_prices.iloc[:3, :min(5, df_prices.shape[1])])

In [ ]:
# Clean and validate the downloaded data
print(f"\n🧹 Cleaning and validating data...")

# Get close prices
if 'Close' in df_prices.columns.names:
    close_prices = df_prices['Close'].copy()
else:
    close_prices = df_prices.copy()

# Remove stocks with too much missing data (>20%)
initial_stock_count = len(close_prices.columns)
missing_threshold = 0.20
stocks_to_remove = []

for stock in close_prices.columns:
    missing_pct = close_prices[stock].isnull().sum() / len(close_prices)
    if missing_pct > missing_threshold:
        stocks_to_remove.append(stock)

if stocks_to_remove:
    print(f"\n⚠️  Removing {len(stocks_to_remove)} stocks with >{missing_threshold*100}% missing data:")
    for stock in stocks_to_remove:
        print(f"   - {stock}")
    close_prices = close_prices.drop(columns=stocks_to_remove)
    STOCK_LIST = [s for s in STOCK_LIST if s not in stocks_to_remove]

# Forward fill remaining missing data
close_prices = close_prices.fillna(method='ffill').fillna(method='bfill')

# Remove stocks with any remaining NaN or zero values
stocks_to_remove_final = []
for stock in close_prices.columns:
    if close_prices[stock].isnull().any() or (close_prices[stock] <= 0).any():
        stocks_to_remove_final.append(stock)

if stocks_to_remove_final:
    print(f"\n⚠️  Removing {len(stocks_to_remove_final)} stocks with invalid data:")
    for stock in stocks_to_remove_final:
        print(f"   - {stock}")
    close_prices = close_prices.drop(columns=stocks_to_remove_final)
    STOCK_LIST = [s for s in STOCK_LIST if s not in stocks_to_remove_final]

# Update the main dataframe
if 'Close' in df_prices.columns.names:
    df_prices = df_prices[[('Close', stock) for stock in close_prices.columns]]
    df_prices.columns = pd.MultiIndex.from_product([['Close'], close_prices.columns])
else:
    df_prices = close_prices

print(f"\n✅ Data cleaning complete!")
print(f"📊 Stocks after cleaning: {len(STOCK_LIST)} (removed {initial_stock_count - len(STOCK_LIST)})")
print(f"📈 Trading days: {len(df_prices)}")
print(f"✓ No missing values: {not df_prices.isnull().any().any()}")
print(f"✓ All prices > 0: {(df_prices > 0).all().all()}")

# Display final stock list
print(f"\n📋 Final stock list ({len(STOCK_LIST)} stocks):")
print(f"   {', '.join(STOCK_LIST)}")

In [ ]:
# Initialize optimizer
optimizer = UltraFastPortfolioOptimizer(
    df=df_prices,
    companies=companies,
    Wi=Wi,
    fund=FUND,
    n_stocks=N_STOCKS,
    batch_size=BATCH_SIZE
)

# Run optimization
best_portfolios, all_results = optimizer.optimize_multiple_returns(
    return_targets=RETURN_TARGETS,
    max_combinations=MAX_COMBINATIONS
)

## 📊 Step 11: Display Results

In [ ]:
# Display beautiful results
display_results(best_portfolios, RETURN_TARGETS, df_prices, FUND, N_STOCKS)

## 💾 Step 12: Save Results to File

In [ ]:
# Save to file
output_filename = f"portfolio_optimization_N{N_STOCKS}_results.txt"
save_results_to_file(best_portfolios, RETURN_TARGETS, N_STOCKS, output_filename)

## 📈 Step 13: Detailed Analysis of Best Portfolios

In [ ]:
# Create summary DataFrame
summary_data = []
for target_idx, (min_ret, max_ret) in enumerate(RETURN_TARGETS):
    portfolio = best_portfolios[target_idx]
    if portfolio['volatility'] is not None:
        summary_data.append({
            'Target Return (%)': (min_ret + max_ret) / 2,
            'Actual Return (%)': portfolio['avg_return'],
            'Volatility': portfolio['volatility'],
            'Variance': portfolio['variance'],
            'Stocks': ', '.join(portfolio['companies']),
            'Status': '✅ Found'
        })
    else:
        summary_data.append({
            'Target Return (%)': (min_ret + max_ret) / 2,
            'Actual Return (%)': None,
            'Volatility': None,
            'Variance': None,
            'Stocks': None,
            'Status': '❌ Not Found'
        })

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*90)
print(f"{'📊 OPTIMIZATION SUMMARY':^90}")
print("="*90 + "\n")
print(summary_df.to_string(index=False))
print("\n" + "="*90)

## 🔍 Step 14: Analyze Single Portfolio in Detail (Optional)

In [ ]:
# Analyze a specific return target in detail
TARGET_TO_ANALYZE = 0  # Index: 0=1%, 1=2%, 2=3%, 3=4%, 4=5%

portfolio = best_portfolios[TARGET_TO_ANALYZE]

if portfolio['volatility'] is not None:
    print(f"\n{'='*90}")
    print(f"{'🔍 DETAILED PORTFOLIO ANALYSIS':^90}")
    print(f"{'Target Return: ' + str(portfolio['target_return']) + '%':^90}")
    print(f"{'='*90}\n")
    
    companies = portfolio['companies']
    weights = portfolio['weights']
    
    # Get price data for these stocks
    if 'Close' in df_prices.columns.names:
        stock_prices = df_prices['Close'][list(companies)]
    else:
        stock_prices = df_prices[list(companies)]
    
    print(f"\n📈 Historical Performance:")
    print(f"   Start Price: {stock_prices.iloc[0].to_dict()}")
    print(f"   End Price:   {stock_prices.iloc[-1].to_dict()}")
    
    # Calculate individual stock returns
    print(f"\n📊 Individual Stock Returns:")
    for stock in companies:
        start_price = stock_prices[stock].iloc[0]
        end_price = stock_prices[stock].iloc[-1]
        stock_return = (end_price - start_price) / start_price * 100
        print(f"   {stock:<15} Return: {stock_return:>8.2f}%")
    
    print(f"\n🎯 Optimal Weights:")
    for stock, weight in zip(companies, weights):
        print(f"   {stock:<15} Weight: {weight:>8.4f} ({weight*100:>6.2f}%)")
    
    print(f"\n" + "="*90)
else:
    print(f"\n❌ No portfolio found for target return {portfolio['target_return']}%")

## 📊 Step 15: Export Results to Excel (Optional)

In [ ]:
# Export to Excel with multiple sheets
excel_filename = f"portfolio_optimization_N{N_STOCKS}_results.xlsx"

with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
    # Summary sheet
    summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    # Detailed results for each target
    for target_idx, (min_ret, max_ret) in enumerate(RETURN_TARGETS):
        portfolio = best_portfolios[target_idx]
        target_return = (min_ret + max_ret) / 2
        
        if portfolio['volatility'] is not None:
            # Create detailed DataFrame
            detail_data = []
            companies = portfolio['companies']
            weights = portfolio['weights']
            
            # Get latest prices
            latest_prices = {}
            for stock in companies:
                try:
                    if 'Close' in df_prices.columns.names:
                        latest_prices[stock] = df_prices['Close'][stock].iloc[-1]
                    else:
                        latest_prices[stock] = df_prices[stock].iloc[-1]
                except:
                    latest_prices[stock] = 0.0
            
            allocation, total_invested, remaining = calculate_allocation(
                weights, FUND, latest_prices
            )
            
            for stock in companies:
                info = allocation[stock]
                detail_data.append({
                    'Stock': stock,
                    'Weight': info['weight'],
                    'Weight (%)': info['weight'] * 100,
                    'Shares': info['shares'],
                    'Price': info['price'],
                    'Invested': info['invested']
                })
            
            detail_df = pd.DataFrame(detail_data)
            sheet_name = f'{target_return:.0f}% Return'
            detail_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"\n💾 Excel file saved: {excel_filename}")

## 🎯 Step 16: Quick Comparison - Different Portfolio Sizes (Optional)

In [ ]:
# This cell allows you to quickly test different portfolio sizes
# Uncomment and run to compare N=2, N=3, N=5

'''
results_comparison = {}

for n in [2, 3, 5]:
    print(f"\n{'='*90}")
    print(f"Testing N={n}")
    print(f"{'='*90}")
    
    # Generate weights
    dist = tfp.distributions.Dirichlet(np.ones(n))
    Wi_test = dist.sample(50000)  # Reduced for speed
    
    # Generate combinations
    companies_test = np.array(list(itertools.combinations(STOCK_LIST, n)))
    
    # Initialize optimizer
    optimizer_test = UltraFastPortfolioOptimizer(
        df=df_prices,
        companies=companies_test,
        Wi=Wi_test,
        fund=FUND,
        n_stocks=n,
        batch_size=50
    )
    
    # Optimize (limit to 500 combinations for quick test)
    best_test, _ = optimizer_test.optimize_multiple_returns(
        return_targets=RETURN_TARGETS[:3],  # Only test 1%, 2%, 3%
        max_combinations=500
    )
    
    results_comparison[n] = best_test

print("\n" + "="*90)
print("Comparison Complete!")
print("="*90)
'''

## 📝 Notes and Tips

### Performance Optimization Tips:
1. **GPU Usage**: Ensure GPU is available for maximum speed (10-50x faster)
2. **Batch Size**: Increase for faster processing (if memory allows)
3. **Weight Samples**: More samples = better results but slower
4. **Portfolio Size**: N=2 is fastest, N=5 is slower but more diversified

### Memory Management:
- Reduce `NUM_WEIGHT_SAMPLES` if running out of memory
- Reduce `BATCH_SIZE` if GPU memory errors occur
- Use `MAX_COMBINATIONS` for testing before full run

### Expected Performance:
- **N=2**: ~1,225 combinations, ~30-60 seconds
- **N=3**: ~19,600 combinations, ~5-10 minutes
- **N=5**: ~2.1M combinations, ~1-3 hours (GPU), ~10-30 hours (CPU)

### Customization:
- Modify `RETURN_TARGETS` for different return levels
- Change `STOCK_LIST` to use different stocks
- Adjust date range for different time periods
- Modify volatility calculation method if needed